# Final Project Phase 2 Summary
This Jupyter Notebook (.ipynb) will serve as the skeleton file for your submission for Phase 2 of the Final Project. Answer all statements addressed below as specified in the instructions for the project, covering all necessary details. Please be clear and concise in your answers. Each response should be at most 3 sentences. Good luck! <br><br>

Note: To edit a Markdown cell, double-click on its text.

## Kendall Roberts

# Data Sources
Include sources (as links) to your datasets. Add any additional data sources if needed. Clearly indicate if a data source is different from one submitted in your Phase I, as we will check that it satisfies the requirements.
*   Downloaded Dataset Source:
https://www.atlantapd.org/home/showpublisheddocument/3051/637141450644130000

#This starts a zip download that contains the csv file: COBRA-2009-2019.csv
Website: https://www.atlantapd.org/i-want-to/crime-data-downloads

*   Web API Collection Source:
This is different than the API one I submitted. I swapped the old source out since the old API had limited imformation that could be provided and the amount of requests I could make was very limited.
https://crime-data-explorer.fr.cloud.gov/pages/docApi


*   Web HTML Collection Source:
This is different than the html one I submitted. I swapped the old source out since the old HTML had limited imformation that could be used.
https://www.zipdatamaps.com/economics/income/agi/metro/wealthiest-zipcodes-in-metro-atlanta



# Imports

In [10]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import re

## Downloaded Dataset Requirement

Fill in the predefined functions with your data scraping/parsing code. You may modify/rename each function as you seem fit, but you must provide at least 3 separate functions that clean each of your required datasets.


In [13]:
def data_parser():
    
    cobra_df = pd.read_csv("COBRA-2009-2019.csv", delimiter = ",", index_col = 0)
    
    cobra_df.drop(["Location Type","Apartment Number","Neighborhood","Apartment Office Prefix","Possible Time","Possible Date"],axis = 1, inplace=True)
    
    #cobra_df.isnull().sum()
    cobra_df.dropna(inplace = True)
    
    cobra_df.rename(columns = {"UCR Literal":"CrimeType","Beat":"PoliceBeat","Location":"StreetAddress"},inplace = True)
    cobra_df.rename(columns = {"Report Date":"ReportDate","Occur Date":"OccurDate","Occur Time":"OccurTime","Shift Occurence":"ShiftOccurence"},inplace = True)
    
    cobra_df[["ReportMonth","ReportDay","ReportYear"]] = cobra_df["ReportDate"].str.split("/", expand = True).astype(int)
    cobra_df[["OccurMonth","OccurDay","OccurYear"]] = cobra_df["OccurDate"].str.split("/", expand = True).astype(int)
    
    cobra_df["ShiftOccurence"] = cobra_df["ShiftOccurence"].map({"Evening Watch":"Evening","Day Watch":"Day","Morning Watch":"Morning"})
    
    cobra_df = cobra_df[~pd.to_numeric(cobra_df["OccurTime"], errors='coerce').isna()]
    
    cobra_df["OccurTime"] = cobra_df["OccurTime"].astype(int)
    
    cobra_df["OccurTimeStandard"] = (cobra_df["OccurTime"] < 1200) | (cobra_df["OccurTime"] == 2400)
    
    cobra_df["AM/PM"] = np.where(cobra_df["OccurTimeStandard"] == True, "AM","PM")
    
    cobra_df["OccurTimeStandard"] = np.where(cobra_df["OccurTime"] <= 1200, cobra_df["OccurTime"], cobra_df["OccurTime"] - 1200)
    
    cobra_df.sort_values(by=["OccurYear", "OccurMonth", "OccurDay"], ascending=True, inplace=True)
    
    print(cobra_df)
    
    cobra_df.to_csv("Cleaned_COBRA-2009-2019.csv", index = True)

############ Function Call ############
data_parser()

C:\Users\spain\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (3,11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


               ReportDate   OccurDate  OccurTime  PoliceBeat  \
Report Number                                                  
160442700       2/13/2016    1/7/1916       1215       412.0   
160902737       3/30/2016   3/29/1916       2300       507.0   
160951996        4/4/2016    4/2/1916       1700       211.0   
161460989       5/25/2016   5/15/1916       2000       206.0   
161592043        6/7/2016   5/30/1916       1400       412.0   
...                   ...         ...        ...         ...   
193652089      12/31/2019  12/31/2019       2030       105.0   
193650336      12/31/2019  12/31/2019        432       206.0   
193650603      12/31/2019  12/31/2019        920       404.0   
193651760      12/31/2019  12/31/2019       1853       404.0   
193652591      12/31/2019  12/31/2019       2045       406.0   

                         StreetAddress ShiftOccurence            CrimeType  \
Report Number                                                                
160442700  

## Downloaded Dataset Explanation
1. Imported the data and saved it as cobra_df

2. Since Apartment Office Prefix, Apartment Number, Location Type, and Neighboorhood had a lot of missing data (332,820),   (274,640), (9,216), (123,630) missing values respectively, I removed these columns since this amount of missing data makes these columns useless. Used cobra_df.isnull().sum() to find out these numbers. Also got rid of Possible Time and Possible Date since these columns seemed redundant or useless.

3. Dropped the rows which contained null data since it would make further operations difficult and there were relatively few of them so not a huge impact on the data sample.

4. Renamed columns: UCR Literal, Beat, Location to CrimeType, PoliceBeat, StreetAddress for more clear column names. Also renamed the column names with spaces to not have any spaces for easier use.

5. In the ShiftOccurence column used .map() to change Evening Watch to Evening, Morning Watch to Morning, Day Watch to Day

6. Checked all the values in the OccurTime to see if they were numeric values and if the value was not numeric then that row containing that value was removed.

7. Converted the OccurTime column values to int type

8. Creates a bool value column called OccurTimeStandard that has True for the value if the value in the OccurTime column is less than 1200 or equal to 2400.

9. Creates a column called AM/PM that uses np.where to put AM if the value in the OccurTime column is True and PM if the value is False.

10. Changes the OccurTimeStandard column back to the OccurTime column values with a cap of 1200 for the occur time to more closely follow the standard time format.

11. Sorted the cobra_df values by OccurYear, OccurMonth, then OccurDay

12. Print out cobra_df then write it to a csv file called Cleaned_COBRA-2009-2019.csv


## Web Collection Requirement \#1API Dataset


In [17]:
def API_parser():
    
    my_key = "Advdd3veLeje4QGRZytKos2jh9B5RsIWwCX1hcLa"
    
    crime_key_list = ["aggravated-assault", "burglary", "larceny", "motor-vehicle-theft", "homicide", "rape", "robbery"]
    
    crime_list = []
    
    column_names=["Actual","Cleared","DataYear","Offence","Agency","State"]
    crime_df = pd.DataFrame()
    
    for crime in crime_key_list:
        r = requests.get("https://api.usa.gov/crime/fbi/sapi/api/summarized/state/GA/" + crime + "/2009/2019?API_KEY=" + my_key)
        data = r.json()
        crime_list.append(data["results"])
    
    temp_list = []
    for list in crime_list:
        temp_list = temp_list + list
    
    for dict in temp_list:
        crime_df = crime_df.append({"Actual":dict["actual"],"Cleared":dict["cleared"],"DataYear":dict["data_year"],
                                    "Offense":dict["offense"],"Agency":dict["ori"],"State":dict["state_abbr"]}, ignore_index = True)
        
    crime_df["Cleared"] = crime_df["Cleared"].astype(int)
    crime_df["Actual"] = crime_df["Actual"].astype(int)
    
    crime_df["ClearedToActualRatio"] = crime_df["Cleared"] / crime_df["Actual"]
    
    crime_df["ClearedToActualRatio"] = crime_df["ClearedToActualRatio"].fillna(0)
    
    crime_df["ClearedPercentage"] = crime_df["ClearedToActualRatio"] * 100
    
    crime_df["Agency"] = crime_df["Agency"].str.replace(r"GA","").astype(int)
    
    crime_df["Offense"] = crime_df["Offense"].str.title()
    
    print(crime_df)
    
    crime_df.to_csv("Cleaned_GA_Crime_Data_2009_2019.csv", index = True)


############ Function Call ############
API_parser()

     Actual  Agency  Cleared  DataYear             Offense State  \
0         1   10000        1    2009.0  Aggravated-Assault    GA   
1        17   10100       14    2009.0  Aggravated-Assault    GA   
2        13   20100        4    2009.0  Aggravated-Assault    GA   
3         1   20200        3    2009.0  Aggravated-Assault    GA   
4         4   30000        2    2009.0  Aggravated-Assault    GA   
..      ...     ...      ...       ...                 ...   ...   
135       0   70200        0    2009.0             Robbery    GA   
136       2   70300        0    2009.0             Robbery    GA   
137      29   80000        3    2009.0             Robbery    GA   
138      17   80100        9    2009.0             Robbery    GA   
139       3   80200        1    2009.0             Robbery    GA   

     ClearedToActualRatio  ClearedPercentage  
0                1.000000         100.000000  
1                0.823529          82.352941  
2                0.307692          30.7692

## Web Collection Requirement #1 API Dataset Explaination
1. Access the API using my key and iterate through the different crime keywords using crime_key_list and converting it to a dictionary.

2. Appends the dictionaries to a list called crime_list then iterates through crime_list to create temp_list which converts the data from a list of lists of dicts into only a list of dicts.

3. Iterates through temp_list and appends the dicts to a pandas dataframe called crime_df with columns: Actual, Cleared, DataYear, Offense, renamed ori to Agency to be more descriptive, and renamed state_abbr to just State. Excluded Data Range column since all values were none. 

4. Created a column called ClearedToActualRatio which is the ratio of cleared cases to actual cases

5. Created a column called ClearedPercentage from the ClearedToActualRatio column that represents the percentage of crimes cleared for that specific crime, agency, and year

6. Gets rid of the GA infront of the values of the Agency column values since that is not need since all the data is from Georgia and converts all the values to int.

7. Converts the values in the Offense Column to uppercase

8. Writes the crime_df to a csv file called Cleaned_GA_Crime_Data_2009_2019.csv


## Web Collection Requirement \#2 HTML Dataset

In [14]:
def web_html_parser():
    r = requests.get("https://www.zipdatamaps.com/economics/income/agi/metro/wealthiest-zipcodes-in-metro-atlanta")
    soup = BeautifulSoup(r.text, "html.parser")
    
    wealth_chart_tag = soup.find("table",{"class":"table table-striped table-bordered table-hover table-condensed"}) 
    tags = wealth_chart_tag.find_all("tr")
    
    column_names=["Rank","Zipcode","ZipName","County","AdjustedGrossIncome","MedianHouseholdIncome"]
    wealth_df = pd.DataFrame(columns = column_names)
    
    for tag in tags[2:]:
        data_tags = tag.find_all("td")
        
        row_list = [tag.text for tag in data_tags if tag.text != "" or tag != None]
        
        if len(row_list) == 10:
            wealth_df = wealth_df.append({"Rank":row_list[0], "Zipcode":row_list[5], "ZipName":row_list[6], "County":row_list[7], "AdjustedGrossIncome":row_list[8], "MedianHouseholdIncome":row_list[9]}, ignore_index = True)
    
    wealth_df["Rank"] = wealth_df["Rank"].astype(int)
    wealth_df["Zipcode"] = wealth_df["Zipcode"].astype(int)
    
    wealth_df["AdjustedGrossIncome"] = wealth_df["AdjustedGrossIncome"].str.replace(r"$","").astype(int)
    wealth_df["MedianHouseholdIncome"] = wealth_df["MedianHouseholdIncome"].str.replace(r"$","").astype(int)
    
    print(wealth_df)
    
    wealth_df.to_csv("Cleaned_Zipcode_Wealth_Data.csv", index = True)
    
    



############ Function Call ############
web_html_parser()

    Rank  Zipcode         ZipName   County  AdjustedGrossIncome  \
0      1    30327         Atlanta   Fulton               563160   
1      2    30305         Atlanta   Fulton               340890   
2      3    30342         Atlanta   Fulton               205630   
3      4    30306         Atlanta   Fulton               204020   
4      5    30326         Atlanta   Fulton               200640   
5      6    30319         Atlanta   DeKalb               198810   
6      7    30309         Atlanta   Fulton               178530   
7      8    30328         Atlanta   Fulton               160570   
8      9    30068        Marietta     Cobb               158120   
9     10    30097          Duluth   Fulton               158110   
10    11    30307         Atlanta   DeKalb               154170   
11    12    30075         Roswell   Fulton               152860   
12    13    30004      Alpharetta   Fulton               151620   
13    14    30338         Atlanta   DeKalb               14584

<ipython-input-14-602d4c32ebbf>:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  wealth_df["AdjustedGrossIncome"] = wealth_df["AdjustedGrossIncome"].str.replace(r"$","").astype(int)
<ipython-input-14-602d4c32ebbf>:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  wealth_df["MedianHouseholdIncome"] = wealth_df["MedianHouseholdIncome"].str.replace(r"$","").astype(int)


## Web Collection Requirement #2 HTML Dataset Explaination
1. Send a request to the Web page I am interested in.Then use bs4 html.parser to obtain a soup object

2. Created a tag called wealth_chart_tag that uses .find() contains the table tag, then uses .find_all() to find all "tr" tags

3. Created a dataframe called wealth_df with columns Rank, Zipcode, ZipName, County, AdjustedGrossIncome, MedianHouseholdIncome.Then iterates through all the tr tags excluding the first two which were empty

4. Used find_all() to find all td tags with the current tr tag and stored it in a value data_tags

5. Created a list comprehension called row_list that converts the tags in data_tags to text if they are not None or ""

6. Adds a row to wealth_df by using .append() mapping the column name to the index in row_list only if that row_list is of the correct length.

7. Converted the Rank and Zipcode columns to int type

8. For the AdjustedGrossIncome and MedianHouseholdIncome columns removed the dollar sign infront of the numbers and then converts the columns to a int type

9. Print out wealth_df then write it to a csv file called Cleaned_Zipcode_Wealth_Data.csv

## Additional Dataset Parsing/Cleaning Functions

Write any supplemental (optional) functions here.

In [ ]:
def extra_source1():
    pass

    
############ Function Call ############
extra_source1()

In [ ]:
# Define further extra source functions as necessary

# Inconsistencies
For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. For the Downloaded Data set in multiple columns there are many null and empty data values. For the columns that had way too many null data values or empty data values for the data to be useful, I dropped those columns. For the columns that had relatively few null or empty values I dropped the row the value was contained in. I used cobra_df.isnull().sum() to count the number of null values in each column so I could decide whether to drop the column or the row. I used cobra_df.dropna(inplace = True) to drop missing values.


2. For the Downloaded Data set the OccurTime column contained mixed data type; mostly possible ints values with a incorrectly entered value with a letter "T" instead of a number. 

    Used cobra_df = cobra_df[~pd.to_numeric(cobra_df["OccurTime"],errors='coerce').isna()]  to check if all the values could be converted to numeric values and if not then the row was removed from cobra_df.
    

3. In the API Web Dataset there was a column for the state abbreviation and in the agency column there was also the state abbreviation infront of the agency number so I removed the unecessary state abbreviation as it was redundant. I use .str.replace() to replace the unecessary values with ""


4. For the Downloaded dataset I renamed confusing and vague columns names like UCR Literal, Beat, and Location to more specific and descriptive names like CrimeType, PoliceBeat, and StreetAddress. I used .rename() to map the old column names to the new column names.


5. I also had to drop multiple columns that had useless or unintersesting data, take uncessary additional words in data values, and sort the data to make more logical sense. I also had to convert numerous data columns to different types and create new columns to allow for more analysis in phase 3. Used .drop(),np.where(), and .astype() and .map()


## Jupyter Notebook Quick Tips
Here are some quick formatting tips to get you started with Jupyter Notebooks. This is by no means exhaustive, and there are plenty of articles to highlight other things that can be done. We recommend using HTML syntax for Markdown but there is also Markdown syntax that is more streamlined and might be preferable. 
<a href = "https://towardsdatascience.com/markdown-cells-jupyter-notebook-d3bea8416671">Here's an article</a> that goes into more detail. (Double-click on cell to see syntax)

# Heading 1
## Heading 2
### Heading 3
#### Heading 4
<br>
<b>BoldText</b> or <i>ItalicText</i>
<br> <br>
Math Formulas: $x^2 + y^2 = 1$
<br> <br>
Line Breaks are done using br enclosed in < >.
<br><br>
Hyperlinks are done with: <a> https://www.google.com </a> or 
<a href="http://www.google.com">Google</a><br>

# Data Collection and Cleaning
You are required to provide data collection and cleaning for the three (3) minimum datasets. Create a function for each of the following sections that reads or scrapes data from a file or website, manipulate and cleans the parsed data, and writes the cleaned data into a new file. 

Make sure your data cleaning and manipulation process is not too simple. Performing complex manipulation and using modules not taught in class shows effort, which will increase the chance of receiving full credit.
